In [1]:
import os
from collections import Counter
import re
import json
import numpy as np

In [2]:
root_data_path = None
root_path = root_data_path

In [3]:
from models import *
from DataGenerator import *
from utils import *
from DataGenerator import *
from preprocess import *
from Encoder import *

Using TensorFlow backend.


In [4]:
import json

In [5]:
os. environ["CUDA_VISIBLE_DEVICES"] = "0"

In [6]:
MAX_ENTITY_NUM = 5

MAX_TITLE_LEN = 30
MAX_SNI_LEN=100
MAX_VERT_NUM=3

In [7]:
attr = ['title',]

In [8]:
news,news_index,category_dict,word_dict,entity_dict = read_news(root_data_path,attr,'entity/News_entity_title.tsv')

In [9]:
news_title, news_vert, news_sni, news_entity=get_doc_input(attr,news, news_index,category_dict,word_dict,entity_dict)

In [10]:
import numpy as np

class NewsFetcher():
    def __init__(self,attr,news_title, news_vert, news_sni,news_entity,onehop):
        self.title = news_title
        self.sni = news_sni
        self.vert = news_vert
        self.entity = news_entity
        self.onehop = onehop
        
        self.attr = attr
    
    def fetch_news(self,docids):
        attr = self.attr
        if 'title' in attr:
            title = self.title[docids]
        else:
            title = None
        if 'vert' in attr:
            vert = self.vert[docids]
        else:
            vert = None
        if 'sni' in attr:
            sni = self.sni[docids]
        else:
            sni = None
        if 'entity' in attr:
            entity = self.entity[docids]
            onehop = self.onehop[entity]

            entity = entity.reshape(list(entity.shape)+[1])
            entity = np.concatenate([entity,onehop],axis=-1)
            if len(entity.shape) == 4:
                entity = entity.reshape((entity.shape[0],entity.shape[1],-1))
            else:
                entity = entity.reshape((entity.shape[0],-1))
            #print(entity.shape,onehop.shape)
            
            return [entity_matrix[entity]]
            
        else:
            entity = None
        
        feature = [title,]
        if entity in attr:
            feature.append(entity)
        
#         for a in attr:
#             feature.append(Table[a])
#         if len(feature) == 1:
#             feature = feature[0]
#         else:
#             feature = np.concatenate(feature,axis=-1)
        
        feature = np.concatenate(feature,axis=-1)
 
        return [feature]

In [11]:
if 'entity' in attr:
    entity_matrix = load_entity_matrix(root_data_path,entity_dict,)
else:
    entity_matrix = None

In [12]:
news_fetcher = NewsFetcher(attr,news_title,news_vert,news_sni,news_entity,None)

In [13]:
embedding_path = None

In [14]:
TrainUserInfo,Train_UserId2Index,Train_UserIndex2Id = parse_train_user(news_index,root_path,'train_users.json')
TestUserInfo,Test_UserId2Index,Test_UserIndex2Id = parse_train_user(news_index,root_path,'test_users.json')
train_docids, train_labels, train_userindex,train_time = parse_train_samples(news_index,Train_UserId2Index,root_path,'train_samples.json')
test_impressions = parse_test_impression(news_index,root_path,'test_sessions.json')
print('finish user and sessions')

finish user and sessions


In [15]:
embedding_matrix, have_word = load_matrix(embedding_path,word_dict)

In [16]:
train_generator = get_hir_train_generator(news_fetcher,TrainUserInfo,Train_UserIndex2Id,train_docids,train_userindex,train_time,train_labels,32)
test_user_generator = get_hir_user_generator(news_fetcher,TestUserInfo,test_impressions,16)
news_generator = get_hir_news_generator(news_fetcher,news_title,16)

In [17]:
def load_entity_matrix(root_data_path,entity_dict):
    
    with open(os.path.join(root_data_path,'entity','entity2id.txt')) as f:
        lines = f.readlines()

    satorid2line = {}

    for i in range(1,len(lines)):
        sid, index = lines[i].strip('\n').split('\t')
        index = int(index)
        satorid2line[sid] = index

    with open(os.path.join(root_data_path,'entity','entity2vec.vec')) as f:
        lines = f.readlines()
        
    stat = 0
    entity_matrix = np.zeros((len(entity_dict)+1,100))
    for e in entity_dict:
        if not sid in satorid2line:
            stat += 1
            continue
        eindex = entity_dict[e]
        index = satorid2line[sid]
        line = lines[index]
        line = line.strip('\n').split('\t')[:-1]
        for j in range(len(line)):
            line[j] = float(line[j])
            entity_matrix[eindex,j] = line[j]
            
    print(stat/len(entity_dict))
    
    return entity_matrix

In [18]:
def parse_test():
    docids = []
    userids = []
    labels = []
    eventimes = []
    
    locs = []
    
    for i in range(len(test_impressions)):
        start = len(docids)
        doc,label,uid, eventime =  test_impressions[i]
        docids+=list(doc)
        labels+=list(label)
        uid = Test_UserId2Index[uid]
        userids +=[uid]*len(label)
        eventimes += [eventime]*len(label)
        ed = len(docids)
        locs.append([start,ed])
        
    docids = np.array(docids)
    userids = np.array(userids)
    eventimes = np.array(eventimes)
    labels = np.array(labels)

    return docids, userids, eventimes, labels,locs

In [19]:
test_docids, test_userids, test_eventimes, test_labels, test_locs = parse_test()

In [20]:
class get_test_generator(Sequence):
    def __init__(self,news_fetcher,user_info,UserIndex2Id, news_id,user_id,imp_time, label, batch_size):
        
        self.news_fetcher = news_fetcher
        
        self.user_info = user_info
        self.UserIndex2Id = UserIndex2Id
        
        self.doc_id = news_id
        self.user_id = user_id
        self.imp_time = imp_time
        self.label = label
        
        self.batch_size = batch_size
        self.Imp = label.shape[0]
        
    def __len__(self):
        return int(np.ceil(self.Imp/self.batch_size))

    def __getitem__(self, idx):
        start = idx*self.batch_size
        ed = (idx+1)*self.batch_size
        if ed >self.Imp:
            ed = self.Imp
        
        doc_ids = self.doc_id[start:ed]
        news_feature = self.news_fetcher.fetch_news(doc_ids)
        
        clicked_ids = []
        for i in range(start,ed):
            uindex = self.user_id[i]
            uid = self.UserIndex2Id[uindex]
            user_click, user_eventime = self.user_info[uid]
            eventime = self.imp_time[i]
            uc = user_click.copy()
            ct = (user_eventime<eventime).sum()
            if ct > 50:
                uc = uc[ct-50:ct]
            else:
                uc = [0]*(50-ct) + uc[:ct]
            clicked_ids.append(uc)
        clicked_ids = np.array(clicked_ids)
        user_feature = self.news_fetcher.fetch_news(clicked_ids)
        
        label = self.label[start:ed]
                
        return (news_feature+user_feature,[label])

In [21]:
test_generator = get_test_generator(news_fetcher,TestUserInfo,Test_UserIndex2Id,test_docids,test_userids,test_eventimes,test_labels[:1250000],32)

In [22]:
def evaluate(plabel,labels,locs):
    AUC = []
    MRR = []
    nDCG5 = []
    nDCG10 =[]
    for i in range(len(locs)):
        start,ed=locs[i]
        score = plabel[start:ed]
        label = labels[start:ed]
        

        auc = roc_auc_score(label,score)
        mrr = mrr_score(label,score)
        ndcg5 = ndcg_score(label,score,k=5)
        ndcg10 = ndcg_score(label,score,k=10)
    
        AUC.append(auc)
        MRR.append(mrr)
        nDCG5.append(ndcg5)
        nDCG10.append(ndcg10)
    AUC = np.array(AUC)
    MRR = np.array(MRR)
    nDCG5 = np.array(nDCG5)
    nDCG10 = np.array(nDCG10)
    
    AUC = AUC.mean()
    MRR = MRR.mean()
    nDCG5 = nDCG5.mean()
    nDCG10 = nDCG10.mean()
    
    return AUC, MRR, nDCG5, nDCG10

In [23]:
def CNN_emb_extractor(length,word_dict,title_word_embedding_matrix):

    sentence_input = Input(shape=(length,), dtype='int32')
    title_word_embedding_layer = Embedding(len(word_dict)+1, 300, weights=[title_word_embedding_matrix],trainable=True)
    
    
    word_vecs0 = title_word_embedding_layer(sentence_input)
    word_vecs1 = Conv1D(300,kernel_size=3,activation='relu',padding='same')(word_vecs0)
    word_vecs2 = Conv1D(300,kernel_size=3,activation='relu',dilation_rate=2,padding='same')(word_vecs1)
    word_vecs3 = Conv1D(300,kernel_size=3,activation='relu',dilation_rate=3,padding='same')(word_vecs2)
        
    word_vecs0 = keras.layers.Reshape((1,length,300))(word_vecs0)
    word_vecs1 = keras.layers.Reshape((1,length,300))(word_vecs1)
    word_vecs2 = keras.layers.Reshape((1,length,300))(word_vecs2)
    word_vecs3 = keras.layers.Reshape((1,length,300))(word_vecs3)

    
    word_vecs = keras.layers.Concatenate(axis=-3)([word_vecs0,word_vecs1,word_vecs2,word_vecs3])
    word_vecs = Dropout(0.2)(word_vecs)
    model = Model(sentence_input,word_vecs)
    
    return model

In [ ]:
def DotLayer():
    import math
    fs = math.sqrt(300)
    vecs_input = Input(shape=(30,600))
    user_vecs = keras.layers.Lambda(lambda x:x[:,:,:300])(vecs_input)
    news_vecs = keras.layers.Lambda(lambda x:x[:,:,300:])(vecs_input)
    inter = keras.layers.Dot(axes=-1)([user_vecs,news_vecs])
    inter = keras.layers.Lambda(lambda x:x/fs)(inter)
    return Model(vecs_input,inter)

In [ ]:
def create_interaction_model(): #with title
    
    user_input = Input(shape=(50,4,30,300))
    news_input = Input(shape=(4,30,300))
    
    news_vecs = keras.layers.Reshape((4*30*300,))(news_input)
    news_vecs = RepeatVector(50)(news_vecs)
    news_vecs = keras.layers.Reshape((50,4,30,300))(news_vecs)
    
    vecs = keras.layers.Concatenate(axis=-1)([user_input,news_vecs])
    dot_layer = DotLayer()
    
    inter_emb = TimeDistributed(TimeDistributed(dot_layer))(vecs)
    inter_vecs = Conv3D(300,kernel_size=(3,3,3),padding='same',activation='relu')(inter_emb)
    
    inter_vec = GlobalMaxPooling3D()(inter_vecs)
    
    vec = Dense(128,activation='relu')(inter_vec)
    vec = Dense(128,activation='relu')(vec)
    score = Dense(1)(vec)
    
    return Model([user_input,news_input,],score)

In [ ]:
def create_model(title_word_embedding_matrix): #with title
    
    emb_extractor = CNN_emb_extractor(30,word_dict,title_word_embedding_matrix)
    inter_model = create_interaction_model()
    
    clicked_title_input = Input(shape=(50,30), dtype='int32')
    clicked_entity_input = Input(shape=(50,max_entity_num,100))
    clicked_one_hop_input = Input(shape=(50,max_entity_num,max_entity_num,100))
    
    title_inputs = Input(shape=(1+npratio,30,),dtype='int32')
    entity_inputs = Input(shape=(1+npratio,max_entity_num,100),dtype='float32')
    one_hop_inputs = Input(shape=(1+npratio,max_entity_num,max_entity_num,100),dtype='float32')


    user_emb = TimeDistributed(emb_extractor)(clicked_title_input)
    news_emb = TimeDistributed(emb_extractor)(title_inputs)
    news_embs = [keras.layers.Lambda(lambda x:x[:,i,])(news_emb) for i in range(1+npratio)]
    
    scores = []
    for i in range(1+npratio):
        score = inter_model([user_emb,news_embs[i]])
        scores.append(score)
    
    scores = keras.layers.Concatenate(axis=-1)(scores)
    logit = keras.layers.Activation('softmax')(scores)
    
    model = Model([clicked_title_input,title_inputs,],logit)
    
    model.compile(loss=['categorical_crossentropy'],
                  optimizer=Adam(lr=0.0001), 
                  metrics=['acc'])
    
    
    one_title_input = Input(shape=(30,), dtype='int32')
    one_entity_input = Input(shape=(max_entity_num,100))
    one_one_hop_input = Input(shape=(max_entity_num,max_entity_num,100))
    
    one_emb = emb_extractor(one_title_input)
    one_score = inter_model([user_emb,one_emb])
    scoring_model = Model([one_title_input,clicked_title_input,],one_score)
    
    
    return model,scoring_model

In [ ]:
for i in range(10):
    model,inter_model = create_model(embedding_matrix)
    model.fit_generator(train_generator)
    plabel = inter_model.predict_generator(test_generator,verbose=1)
    g = evaluate(plabel[:,0],test_labels, test_locs[:20000])
    s = json.dumps(g)
    with open('FIM.json','a') as f:
        f.write(s+'\n')

Epoch 1/1
39063/39063 [==============================] - 2989s 77ms/step
Epoch 1/1
39063/39063 [==============================] - 2978s 76ms/step
Epoch 1/1
7389/7448 [============================>.] - ETA: 22s - loss: 0.6096 - acc: 0.6631